# Data Quality Monitoring with Autoencoders at CMS

<div class="alert alert-block alert-info">
  <b>Credits:</b> A. Papanastassiou (INFN-Firenze), B. Camaiani (INFN-Firenze)<br>
    Fifth ML-INFN Hachathon, <a href=https://agenda.infn.it/event/37650/overview> link </a>
</div>

## Introduction

### The CMS experiment

The Compact Muon Solenoid (CMS) experiment  is one of the four main particle detectors located at the Large Hadron Collider (LHC) at CERN. It is  a multi-purpose experiment, designed to study high-energy proton-proton collisions to better understand the fundamental forces and particles that make up the universe.

The CMS apparatus is composed of a complex system of sub-detectors to detect electrons, photons, muons and hadrons.

<div align="center">
<img src="https://alpapana.web.cern.ch/sezione_cms.jfif" width="400" height="400" />
</div>


The only particles that CMS can not directly detect are neutrinos, because they interact very weakly with matter and for this they escape from the detection.
To indirectly observe neutrinos, a kinematics observable called ***missing transverse energy*** (**MET**) is usually employed. MET is defined as:


$$ \text{MET} =  \big|  - \sum_i \vec{p}_{T,i}\big| $$

where $\vec{p}_{T,i}$ is the transverse momentum of the $i$-th reconstructed particle of the final state.
Since the transverse momentum of the initial state is null, according to the law of conservation of momentum and energy, MET is expected to vanish if all products of a collision were detected. However, because neutrinos and other weakly interacting particles can escape the detector without being directly detected, their presence result in a non vanishing missing transverse energy value.


Particles that have a colour charge (like quarks and gluons) can not be directly observed as well. This is because a fundamental principle called *color confinement*, according to which colour charged particles can not be isolated and they always combine in ways that ensure their overall colour charge is color neutral. In order to obey colour confinement, quarks and gluons produced in strong interaction processes create other colored particles to form hadrons clustered in ***jets***, i.e. collimated *groups of colorless objects*.

<div align="center">
<img src="https://alpapana.web.cern.ch/Sketch_PartonParticleCaloJet.png" width="400" height="400" />
</div>

LHC is a proton-proton collider, where protons are injected inside the accelerator: each time this happens it is called "fill". CMS operates during these fills and during each of them data is gathered in "luminosity sections", lumisections in short (LSs), that are sub-sections corresponding to around 23 seconds of data taking during which the instantaneous luminosity is almost constant. LSs are grouped in runs, of thousands of LSs. 

### Data Quality Monitoring (DQM)

Being CMS composed of various subsystems, each serving a specific purpose in particle detection and measurement, issues in the different sub-detectors can arise due to various factors, such as radiation damage, electronic noise, and aging of components. The monitoring of data quality is therefore crucial both online, during the data taking, to promptly spot issues and act on them, and offline, to provide analysts with datasets that are cleaned against the occasional failures that may have crept in.

We focus on the offline side, called Data Certification (DC): the final step of quality checks performed by DQM on recorded collision events. We look at integrated (over the whole run) quantities ("Monitor Elements") pertaining to hadronic jets and MET.




### Monitor Elements (MEs)

Many different quantities related to MET and jets (collectively called JME) are monitored to accertain the quality of data. One of them is MET itself, another one is ***MET Significance*** defined as

$$ \text{METSig} =  \frac{\text{MET}}{\sqrt{\sum_i |\vec{p}_{T,i}|}}\,\,\, . $$

Other quantities are associated with jets, for example the Jetmass. 

Anomalies can be visible in one or more of these MEs and usually are spotted by eye by operators instructed on histograms of "healty" or GOOD data. In the picture you can see an example of GOOD (green) and BAD (blue and orange) runs from the point of view of MET Significance.


<div align="center">
<img src="https://alpapana.web.cern.ch/anomalous_runs.png" width="400" height="400" />
</div>

### Per-Lumisection data


For the specific case of quantities pertaining to hadronic jets and missing transverse momentum (MET), an issue in a few LSs would cause the entire run to be flagged as problematic (***BAD***), and thus removed from the pool of "good-for-analysis" data (***GOOD***).

The possibility of accumulating quantities monitored for data quality purposes per-LS has been recently extended to JME MEs.
This possibility allows for a higher granularity detection of anomalies, potentially enabling the saving of higher amounts of data from runs presenting only a limited set of anomalous LSs. Given the high number, 𝑂(1000), of LSs to be analyzed for each run, an automated approach (rather than a manual one) for DC is required.  
Machine Learning (ML), particularly Neural Networks (NN), can be implemented to this end.
An unsupervised ML model based on a specific NN architecture called ***AutoEncoder*** (**AE**) is employed.


### Autoencoders for anomaly detection

Autoencoders are a particular kind of unsupervised neural network capable of learning efficient codings of unlabelled data. The model is composed of an encoder and a decoder, the first one brings data to a representation of lower dimesionality (latent space), the second one brings it back to the original dimension.

During training, the task is to **reconstruct the input approximately, preserving only the most relevant aspects of it**. The reconstruction loss quantifies the distance between input and output and must be minimized during training. What prohibits the Autoencoder to learn the Identity between input and output is its own architecture: a bottleneck for information provided e.g. by a decrease of the nodes number in the inner layers.


<div align="center">
<img src="https://alpapana.web.cern.ch/Autoencoder_working0.png" width="500"/>
</div>

For anomaly detection purpuses, the autoencoder is trained on non-anomalous data. If an anomaly is present in the testing set, it will result in a peak in the reconstruction loss.

In the context of DQM, the Autoencoder is trained on ordered sequences of histograms related to a specific ME, each representing one LS, coming from a GOOD run. Then we will test the model on per-LS data coming from a run flagged BAD, meaning that some MEs, integrated over the run, show an uncommon shape. By looking at the reconstruction loss we will hopefully be able to tell if the anomaly affecting the run is restricted to a **limited set of LSs**. 

<div align="center">
<img src="https://alpapana.web.cern.ch/Autoencoder_working_.png" width="500"/>
</div>


### Luminosity dependence

During each run the istantaneous **luminosity decreases** and this brings to an appreciable evolution of the MEs observed per-lumisection. In particular a clear reduction in the magnitude of each quantity is visible. Being this is an unavoidable effect, we want our models to be able to spot anomalies occuring in every luminosity condition, therefore we would like to account for the luminosity decrease by integrating the luminosity information inside the model. There are many possible ways to do this but the easier one is to add the luminosity value of each LS to the input of the autoencoder both during training and testing. In this exercise we will not include this input because for the particular anomalies presented the model is powerful enough to complete the task without information about the luminosity of each LS.

## Let's start!

### Load, read and understand the data

Let's start the hackathon by reading the dataset and understanding what we will work with. We are going to use sequences of histograms related to the Monitor Element METSig loaded from a `.csv` file

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
import tensorflow as tf
import keras as keras
from keras import layers

First, we will download the data samples for this hands-on:

In [ ]:
if not os.path.isdir('data'):
    os.makedirs('data')
    os.system("wget -q -O data/data.zip 'https://www.dropbox.com/scl/fo/ppvqy9lrz9be3zkgwtgmp/AKM_bLVCT99aMe58ME5dNp8?rlkey=ogogm7y1k7q3qb177tvofxz72&st=fkysjqlj&dl=0' && unzip data/data.zip -d data/ 2>/dev/null")
    os.remove("data/data.zip")
else:        
    print("Data folder does exist!")

We are going to load the dataset containing the GOOD run to be used as training for the Autoencoder

In [ ]:
csv='data/run_gggggg_METSig.csv'

In [ ]:
#Read the .csv file and convert it to pandas dataframe
df=pd.read_csv(csv,names=range(2),sep=';') 

In [ ]:
df

The dataset contains around a thousand and an half of entries: each one is composed by a LS number (first column) and the content of the corresponding histogram (second column).\
We are going to extract the sequence of histograms necessary to train the Autoencoder: first we order the dataset based on the lumisection number, then we extract the column containing the histograms and we unstack them based on the presence of the comma.

In [ ]:
df=df.drop([0])
df[0] = df[0].astype(int)
df_s=df.sort_values(by=[0],ignore_index=True)  # ordering by "fromlumi"
df_bins=df_s[1].str.extractall('(\d+)')[0].unstack()   # extract and unstack
df_bins_train=df_bins.reset_index(drop=True)

We want data for training to begin with non-zero values, so we are going to remove the first possibly empty lumisections. `zeros` would be the number of empty LSs.

In [ ]:
#identiying empty LSs.

x_train=np.array(df_bins, dtype=np.float64)  # convert df to numpy array
zeros=0
for i in range(x_train.shape[0]):
    if np.all(x_train[i,:]==0):
        zeros+=1
    else:
        break

In [ ]:
zeros

In [ ]:
#remove empty LSs
x_train_=x_train[zeros:,:]

In [ ]:
x_train_

### Preprocessing

Before feeding the model with training data we want to make it more manageble via a rescaling in the [0,1] interval, this is a common practice for this kind of models. Different rescalings are possible, but one that we found very effecive is the following bin by bin rescaling:


In [ ]:
min_val = np.min(x_train_,axis=0)
max_val = np.max(x_train_,axis=0)
data_= np.divide(x_train_, max_val - min_val, out=np.zeros_like(x_train_), where=max_val - min_val!=0)

In [ ]:
data_

### Plot the entire training run

To see the histogram of the whole run we sum over all the LSs. This is what usually is presented to people who perform data certification. In this case the ME is not showing any kind of anomalous shape.

In [ ]:
l=np.sum(x_train_, axis=0)
plt.plot(l,ds = 'steps-mid',linewidth=1)
plt.yscale("log")

### Check luminosity dependence

To see the luminosity dependence in the run you can try the following script, it will plot the whole sequence of LSs histograms. Notice how the bin content decreases during the run

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

m = x_train.shape[0]

fig, ax = plt.subplots()
line, = ax.plot(x_train[0, :], drawstyle='steps-mid', linewidth=1)
ax.set_xlabel('METSIg')
ax.set_title('LS=1')

# Fix limits once so the canvas doesn't rescale each frame
ax.set_xlim(0, x_train.shape[1] - 1)
ymax = x_train.max()
ax.set_ylim(0, ymax if ymax > 0 else 1)

def update(i):
    line.set_ydata(x_train[i, :])
    ax.set_title(f'LS={i+1}')
    return (line,)

ani = animation.FuncAnimation(fig, update, frames=m, interval=50, blit=True)

plt.close(fig)  # prevents duplicate static figure display
HTML(ani.to_jshtml())


## Autoencoder model definition

We are now going to define our model. This will be a dense Autoencoder, meaning a neural network consisting of dense layers in which the encoding of information and subsequent decoding is obtained via a decrease (encoding) and following increase (decoding) of the number of nodes in the layers. This way the encoded representation inside the latent layer has a lower dimensionality and only the most relevant features can be preserved.



### Dense Undercomplete Autoencoder

<div align="left">
<img src="https://alpapana.web.cern.ch/UnderDense.PNG" width="300"/>
</div>

Practice with the code and after the first layer add a 2 layers for encoding and the required number of layers for decoding the information. Last layer should have a specific shape. Activation functions should be `relu` for the hidden layers and `sigmoid` for the output layer.

In [ ]:
training=data_
input = keras.Input(shape=(training.shape[1],))
learning_rate = 1e-7
encoding_dim = 103
encoding_dim_2 = 82

# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu',activity_regularizer=tf.keras.regularizers.l2(learning_rate))(input)
latent = layers.Dense(encoding_dim_2, activation='relu')(encoded)
decoded = layers.Dense(encoding_dim, activation='relu')(latent)
decoded = layers.Dense(training.shape[1], activation='sigmoid')(decoded) # "decoded" is the lossy reconstruction of the input
metrics = ['mse']
model = keras.Model(input, decoded)
model.compile(optimizer='adam', loss='mse', metrics=metrics)

### Train the model 

To train the model we define a batch size and the number of epochs

In [ ]:
batch_size =42
epochs = 150
history=model.fit(training, training, batch_size = batch_size, epochs = epochs,
          validation_data=(training,training),shuffle=False)

### Plot the loss

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('MSE')

## Test the model on a BAD run

### Load test run

We define a function to repeat the operations performed earlier on the testing dataset, that is data coming from a run flagged BAD, for the same ME (METSig)

In [ ]:
def build_test_dataset(csv):
    df=pd.read_csv(csv,names=range(15),sep=';')
    df=df.drop([0])
    df[0] = df[0].astype(int)
    df_s=df.sort_values(by=[8],ignore_index=True)
    df_bins=df_s[1].str.extractall('(\d+)')[0].unstack()
    df_bins_train=df_bins.reset_index(drop=True)
    x_train=np.array(df_bins, dtype=np.float64)
    zeros=0
    for i in range(x_train.shape[0]):
        if np.all(x_train[i,:]==0):
            zeros+=1
        else:
            break
    x_train_=x_train[zeros:,:]
    #x_train[155]=x_train[155]+(x_train[70,:]-x_train[71])//3
    #x_train[155]=x_train[155]+np.array([  0.,   0.,   0.,   0.,   0.,   0.,   0., 1.,  7.,  17.,  13.,  27.,  39.,  57.,  63.,  73.,  80., 86., 115., 153., 210., 337., 426., 472., 404., 313., 211., 128., 73.,  47.,  22.,  14.,   4.,   2.,   0.,   0.,   0.,   0., 0., 0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 0.,   0.,   0.])
    min_val = np.min(x_train_,axis=0)
    max_val = np.max(x_train_,axis=0)
    data_= np.divide(x_train_, max_val - min_val, out=np.zeros_like(x_train_), where=max_val - min_val!=0)
    return zeros, data_, df_s, x_train, df_bins_train

#zeros: the number of empty LSs
#data_: the dataset rescaled (as np array)
#df_s: the dataset before the extraction of the histograms (contains the LS number)
#x_train: the dataset before rescaling (as np array)

In [ ]:
csv='data/run_b1bbbb_METSig.csv'

In [ ]:
zeros, testing, df_s, x_test, df_bins_test=build_test_dataset(csv)

In [ ]:
zeros

### Use the trained model to check the run for anomalies

In [ ]:
test_predictions=model.predict(testing)

mse_dense = tf.math.reduce_mean(tf.math.pow(testing - test_predictions, 2), axis = 1)

In [ ]:
plt.plot(mse_dense)
plt.xlabel('LS')
plt.ylabel('MSE')
plt.title('MSE between input and output of the AE for testing data')

* How many anomalies you think are present?

## Quantitative analysis of the MSE

We can select the higher peaks by calculating the 99.x percentile of the reconstruction loss or of a moving average of the loss, as in the following code

In [ ]:
def moving_average(a, n=1) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def anomalies_finder(mse_dense, percentile, df_s, zeros, n):
    data=moving_average(mse_dense, n)
    #your code here...
    d = np.percentile(data, percentile)
    anomalies=np.where(mse_dense>d)[0]

    LS=[]
    for i in range(anomalies.shape[0]):
        LS.append(df_s[0][anomalies[i]+zeros])
    print(LS)
    return LS

In [ ]:
LS=anomalies_finder(mse_dense, 99.5, df_s, zeros, n=3)

* How many anomalies at 99.5 percentile?

Let's try to plot the run with and without the identified anomalous LSs. This will help us verifying if these are significant anomalies

In [ ]:
#sum over all the LSs of the non rescaled data (x_test)
l=np.sum(x_test, axis=0)
#subtract from x_test the anomalous LSs and sum over the remaining LSs
cleaned_x_test =np.delete(x_test, [x-zeros for x in df_s.index[df_s[0].apply(lambda x: x in LS)].tolist()], axis=0)
s=np.sum(cleaned_x_test, axis=0)

In [ ]:
#Plot l and s and check if the anomalous LSs have disapeared
plt.plot(l,ds = 'steps-mid',linewidth=1, label='uncleaned test run')
plt.plot(s,ds = 'steps-mid',linewidth=1, label='first cleaned test run')
plt.yscale("log")
plt.legend()
plt.xlabel('METSig')

In [ ]:
LS_2=anomalies_finder(mse_dense, 99.9, df_s, zeros, n=3)

In [ ]:
cleaned_x_test_2 =np.delete(x_test, [x-zeros for x in df_s.index[df_s[0].apply(lambda x: x in LS_2)].tolist()], axis=0)
s_2=np.sum(cleaned_x_test_2, axis=0)

In [ ]:
#Plot l and s and check if the anomalous LSs have disapeared
plt.plot(l,ds = 'steps-mid',linewidth=1, label='uncleaned test run')
plt.plot(s_2,ds = 'steps-mid',linewidth=1, label='second cleaned test run')
plt.plot(s,ds = 'steps-mid',linewidth=1, label='first cleaned test run')

plt.yscale("log")
plt.legend()
plt.xlabel('METSig')

### Final check

Have a look at the whole run, you can spot the anomalies by eye ...

In [ ]:
m = x_test.shape[0]

fig, ax = plt.subplots()
line, = ax.plot(x_test[0, :], drawstyle='steps-mid', linewidth=1)
ax.set_xlabel('METSIg')
ax.set_title('LS=1')

# Fix limits once so the canvas doesn't rescale each frame
ax.set_xlim(0, x_test.shape[1] - 1)
ymax = x_test.max()
ax.set_ylim(0, ymax if ymax > 0 else 1)

def update(i):
    line.set_ydata(x_test[i, :])
    ax.set_title(f'LS={i+1}')
    return (line,)

ani = animation.FuncAnimation(fig, update, frames=m, interval=50, blit=True)

plt.close(fig)  # prevents duplicate static figure display
HTML(ani.to_jshtml())

## Another BAD run to test

With the model trained we can test it on another anomalous run. for the sake of clarity we will refer to it as run3

In [ ]:
csv='data/run_b2bbbb_METSig.csv'

In [ ]:
zeros_3, testing_3, df_s_3, x_test_3, df_bins_test_3=build_test_dataset(csv)

In [ ]:
zeros, testing, df_s, x_test, df_bins_test=build_test_dataset(csv)

In [ ]:
test_predictions_3=model.predict(testing_3)

mse_dense_3 = tf.math.reduce_mean(tf.math.pow(testing_3 - test_predictions_3, 2), axis = 1)
mae_dense_3 = tf.math.reduce_mean(tf.math.abs(testing_3 - test_predictions_3), axis = 1)

In [ ]:
plt.plot(mse_dense_3)
plt.xlabel('LS')
plt.ylabel('MSE')
plt.title('MSE between input and output of the AE for testing data run3')

Again we want to select the most relevant peaks

In [ ]:
LS_3=anomalies_finder(mse_dense_3, 99.9, df_s_3, zeros_3, n=10)

In [ ]:
l_3=np.sum(x_test_3, axis=0)
cleaned_x_test_3 =np.delete(x_test_3, [x-zeros_3 for x in df_s_3.index[df_s_3[0].apply(lambda x: x in LS_3)].tolist()], axis=0)
s_3=np.sum(cleaned_x_test_3, axis=0)

In [ ]:
#Plot l and s and check if the anomalous LSs have disapeared
plt.plot(l_3,ds = 'steps-mid',linewidth=1, label='uncleaned test run')
plt.plot(s_3,ds = 'steps-mid',linewidth=1, label='cleaned test run')

plt.yscale("log")
plt.legend()
plt.xlabel('METSig')

It seems like there is another anomaly the Autoencoder was not able to detect or it corresponds to one of the peaks but it is not high enough to be distinguished.\
One possible way out is to try modyfing the structure adding layers or changing hyperparameters. Do it as homework. You can also try to test with another function like `mae`. Is there an improvement? \
Another way is to try a different Autoencoder model...